## オートエンコーダ(AE)による特徴量エンジニアリング

特徴量セットB (`df_processed_b`) をAEに入力し、低次元の特徴量表現を獲得します。
TensorFlow (Keras) が必要です: pip install tensorflow


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import time

# --- 1. データの読み込み ---
print("--- 1. CSVデータの読み込み ---")
csv_filename = 'BTCUSDT_5m_processed_set_b.csv'
try:
    df_processed_b_loaded = pd.read_csv(csv_filename, index_col='timestamp', parse_dates=True)
    print(f"'{csv_filename}' を読み込みました。")
except FileNotFoundError:
    print(f"エラー: ファイル '{csv_filename}' が見つかりません。")
    df_processed_b_loaded = None
except Exception as e:
    print(f"CSV読み込み中にエラーが発生しました: {e}")
    df_processed_b_loaded = None

# --- 2. データ分割 ---
if df_processed_b_loaded is not None and not df_processed_b_loaded.empty:
    print("\n--- 2. データ分割 ---")
    exclude_cols = ['open', 'high', 'low', 'close', 'volume', 'turnover', 'wclprice', 'target']
    features_b = [col for col in df_processed_b_loaded.columns if col not in exclude_cols]
    n_original_features = len(features_b)
    print(f"元の特徴量の数: {n_original_features}")

    X_b = df_processed_b_loaded[features_b]
    y_b = df_processed_b_loaded['target'] # y はAE学習には使わないが、後で使う

    test_size = 0.2
    X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
        X_b, y_b, test_size=test_size, shuffle=False
    )
    print(f"学習データ数: {len(X_train_b)}, テストデータ数: {len(X_test_b)}")

    # --- 3. データ準備 (スケーリング) ---
    print("\n--- 3. データ準備 (スケーリング) ---")
    scaler_ae = StandardScaler() # AE用に新しいスケーラー
    X_train_scaled = scaler_ae.fit_transform(X_train_b)
    X_test_scaled = scaler_ae.transform(X_test_b)
    print("スケーリング完了。")

    # --- 4. オートエンコーダ(AE)モデルの定義 ---
    print("\n--- 4. オートエンコーダモデル定義 ---")
    encoding_dim = 16  # 圧縮後の次元数 (ハイパーパラメータ、元の66次元から削減)
    print(f"エンコーディング次元数: {encoding_dim}")

    # 入力層
    input_layer = keras.Input(shape=(n_original_features,))

    # エンコーダ部分
    encoded = layers.Dense(64, activation='relu')(input_layer)
    encoded = layers.BatchNormalization()(encoded)
    encoded = layers.Dropout(0.1)(encoded)
    encoded = layers.Dense(32, activation='relu')(encoded)
    encoded = layers.BatchNormalization()(encoded)
    encoded = layers.Dropout(0.1)(encoded)
    encoder_output = layers.Dense(encoding_dim, activation='relu', name='encoder_output')(encoded) # ボトルネック層

    # デコーダ部分
    decoded = layers.Dense(32, activation='relu')(encoder_output)
    decoded = layers.BatchNormalization()(decoded)
    decoded = layers.Dropout(0.1)(decoded)
    decoded = layers.Dense(64, activation='relu')(decoded)
    decoded = layers.BatchNormalization()(decoded)
    decoded = layers.Dropout(0.1)(decoded)
    decoded = layers.Dense(n_original_features, activation='linear')(decoded) # 出力層は元の次元数、活性化は線形

    # オートエンコーダモデル (入力から再構築まで)
    autoencoder = Model(input_layer, decoded, name="autoencoder")
    # エンコーダモデル (入力からボトルネックまで)
    encoder = Model(input_layer, encoder_output, name="encoder")

    autoencoder.summary()

    # --- 5. AEモデルのコンパイルと学習 ---
    print("\n--- 5. AEモデルのコンパイルと学習 ---")
    autoencoder.compile(optimizer='adam', loss='mse') # 損失関数は再構築誤差(MSE)

    epochs_ae = 50 # AEの学習エポック数 (様子を見て調整)
    batch_size_ae = 512

    # EarlyStopping: 検証損失(val_loss)が改善しなくなったら停止
    early_stopping_ae = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=5,
                                                      mode='min',
                                                      restore_best_weights=True)

    start_ae_train_time = time.time()
    print("AE学習中...")
    history_ae = autoencoder.fit(X_train_scaled, X_train_scaled, # 入力と出力(目標)が同じ
                                 epochs=epochs_ae,
                                 batch_size=batch_size_ae,
                                 shuffle=True, # AEの学習ではシャッフルして良い
                                 validation_data=(X_test_scaled, X_test_scaled), # テストデータで再構築誤差を評価
                                 callbacks=[early_stopping_ae],
                                 verbose=1)
    end_ae_train_time = time.time()
    print(f"AE学習完了。所要時間: {end_ae_train_time - start_ae_train_time:.2f} 秒")
    print(f"最終的な検証損失 (Val Loss): {history_ae.history['val_loss'][-1]:.6f}")

    # --- 6. 特徴量の変換 ---
    print("\n--- 6. エンコーダによる特徴量変換 ---")
    # 学習済みエンコーダを使って、学習・テストデータを低次元表現に変換
    X_train_ae = encoder.predict(X_train_scaled)
    X_test_ae = encoder.predict(X_test_scaled)

    print(f"変換後の学習データ形状: {X_train_ae.shape}") # (サンプル数, encoding_dim)
    print(f"変換後のテストデータ形状: {X_test_ae.shape}")   # (サンプル数, encoding_dim)
    print("特徴量変換完了。")

    # これで X_train_ae と X_test_ae が新しい特徴量セットとして準備できました。
    # 次のステップで、これらと y_train_b, y_test_b を使って予測モデル(MLPやLightGBM)を学習させます。

else:
    print(f"データフレーム df_processed_b_loaded の準備に失敗したか、'{csv_filename}' が見つかりませんでした。")

--- 1. CSVデータの読み込み ---
'BTCUSDT_5m_processed_set_b.csv' を読み込みました。

--- 2. データ分割 ---
元の特徴量の数: 59
学習データ数: 426804, テストデータ数: 106701

--- 3. データ準備 (スケーリング) ---
スケーリング完了。

--- 4. オートエンコーダモデル定義 ---
エンコーディング次元数: 16


Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 59)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_output (Dense)          │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 59)             │         3,835 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,707 (53.54 KB)

 Trainable params: 13,323 (52.04 KB)

 Non-trainable params: 384 (1.50 KB)


--- 5. AEモデルのコンパイルと学習 ---
AE学習中...
Epoch 1/50
Epoch 1/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.5829 - val_loss: 0.3515
Epoch 2/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2156 - val_loss: 0.3381
Epoch 3/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1903 - val_loss: 0.3059
Epoch 4/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1745 - val_loss: 0.3190
Epoch 5/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1597 - val_loss: 0.3210
Epoch 6/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1507 - val_loss: 0.3253
Epoch 7/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1471 - val_loss: 0.3144
Epoch 8/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1413 - val_loss: 0.2745
Epoch 9/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1349 - val_loss: 0.2756
Epoch 10/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1321 - val_loss: 0.2423
Epoch 11/50
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1296 - val_loss: 0.2126
Epoch

## AE特徴量を用いた予測モデル (LightGBM) の学習・評価

オートエンコーダによって生成された特徴量 (`X_train_ae`, `X_test_ae`) を使用して、
前回チューニングした最適パラメータで LightGBM モデルを学習・評価します。

In [3]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np
import time

# AE特徴量 (X_train_ae, X_test_ae) とターゲット (y_train_b, y_test_b) が存在すると仮定
# チューニング済みパラメータ best_params_b も存在すると仮定 (前回結果より)
if 'X_train_ae' in locals() and 'y_train_b' in locals() and \
   'X_test_ae' in locals() and 'y_test_b' in locals():

    # 前回確認した Set B チューニング時の最適パラメータを使用
    best_params_b = {'num_leaves': 183, 'learning_rate': 0.050569794707552296, 'max_depth': 5, 'reg_alpha': 4.462590221186446e-08, 'reg_lambda': 1.4306490938848109e-05, 'colsample_bytree': 0.626063327690084, 'subsample': 0.9387972827397255, 'n_estimators': 237}
    print("--- モデル学習 (AE特徴量 + チューニング済みLGBM) ---")
    print("使用するパラメータ:", best_params_b)

    # 最適パラメータでモデルを初期化
    model_ae_lgbm = lgb.LGBMClassifier(**best_params_b, random_state=42, n_jobs=-1)

    # 学習時間計測
    start_train_time = time.time()
    print("学習中 (AE特徴量使用)...")
    # AE特徴量でフィット
    model_ae_lgbm.fit(X_train_ae, y_train_b)
    end_train_time = time.time()
    print(f"学習完了。所要時間: {end_train_time - start_train_time:.2f} 秒")

    # --- テストデータでの評価 ---
    print("\n--- テストデータでの評価 (AE特徴量 + チューニング済みLGBM) ---")
    y_pred_ae_lgbm = model_ae_lgbm.predict(X_test_ae)
    y_pred_proba_ae_lgbm = model_ae_lgbm.predict_proba(X_test_ae)[:, 1]

    accuracy_ae_lgbm = accuracy_score(y_test_b, y_pred_ae_lgbm)
    auc_ae_lgbm = roc_auc_score(y_test_b, y_pred_proba_ae_lgbm)

    print(f"テストデータ Accuracy: {accuracy_ae_lgbm:.4f}")
    print(f"テストデータ AUC Score: {auc_ae_lgbm:.4f}") # ★AE特徴量の効果を見る指標
    print("\nClassification Report:")
    print(classification_report(y_test_b, y_pred_ae_lgbm, target_names=['Low (0)', 'High (1)']))
    print("\nConfusion Matrix:")
    conf_matrix_ae_lgbm = confusion_matrix(y_test_b, y_pred_ae_lgbm)
    conf_matrix_ae_lgbm_df = pd.DataFrame(conf_matrix_ae_lgbm, index=['Actual Low', 'Actual High'], columns=['Predicted Low', 'Predicted High'])
    display(conf_matrix_ae_lgbm_df)

    # 結果を保存
    if 'model_results' not in locals(): model_results = {}
    model_results['LGBM_Tuned_on_AE_Features'] = {
        'model': model_ae_lgbm,
        'auc': auc_ae_lgbm,
        'accuracy': accuracy_ae_lgbm,
        'y_pred_proba': y_pred_proba_ae_lgbm,
        'features': 'Set B -> AE (dim=16)' # 特徴量情報を記録
    }

else:
    print("AE特徴量またはターゲットデータが見つかりません。前のステップを実行してください。")

--- モデル学習 (AE特徴量 + チューニング済みLGBM) ---
使用するパラメータ: {'num_leaves': 183, 'learning_rate': 0.050569794707552296, 'max_depth': 5, 'reg_alpha': 4.462590221186446e-08, 'reg_lambda': 1.4306490938848109e-05, 'colsample_bytree': 0.626063327690084, 'subsample': 0.9387972827397255, 'n_estimators': 237}
学習中 (AE特徴量使用)...
[LightGBM] [Info] Number of positive: 210247, number of negative: 216557
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005995 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4080
[LightGBM] [Info] Number of data points in the train set: 426804, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.492608 -> initscore=-0.029571
[LightGBM] [Info] Start training from score -0.029571
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,Predicted Low,Predicted High
Actual Low,37711,15638
Actual High,36434,16918


## AE特徴量を用いた予測モデル (MLP) の学習・評価

オートエンコーダによって生成された16次元の特徴量 (`X_train_ae`, `X_test_ae`) を使用して、
MLPモデルを学習・評価します。ターゲット変数は元の `y_train_b`, `y_test_b` を使用します。
TensorFlow (Keras) が必要です。


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import time

# AE特徴量 (X_train_ae, X_test_ae) とターゲット (y_train_b, y_test_b) が存在すると仮定
if 'X_train_ae' in locals() and 'y_train_b' in locals() and \
   'X_test_ae' in locals() and 'y_test_b' in locals():

    print(f"入力特徴量の数 (AE): {X_train_ae.shape[1]}")

    # --- MLPモデルの定義 (前回と同様の構造、入力次元のみ変更) ---
    print("\n--- MLPモデル定義 (AE特徴量用) ---")
    n_ae_features = X_train_ae.shape[1] # AE特徴量の次元数 (16)
    tf.random.set_seed(42) # 再現性のため乱数シード固定

    # スケーリングはAE学習前に行っているので、AE特徴量には不要な場合が多い
    # 必要であれば再度スケーリングを検討
    # scaler_mlp_ae = StandardScaler()
    # X_train_ae_scaled = scaler_mlp_ae.fit_transform(X_train_ae)
    # X_test_ae_scaled = scaler_mlp_ae.transform(X_test_ae)
    # 使用するデータはスケーリング済みとする (AEへの入力がスケーリング済みだったため)
    X_train_input = X_train_ae
    X_test_input = X_test_ae


    model_mlp_ae = keras.Sequential(
        [
            keras.Input(shape=(n_ae_features,)), # 入力次元をAE特徴量の次元に合わせる
            layers.BatchNormalization(),
            layers.Dense(128, activation="swish", kernel_regularizer=keras.regularizers.l2(0.001)),
            layers.Dropout(0.3),
            layers.BatchNormalization(),
            layers.Dense(64, activation="swish", kernel_regularizer=keras.regularizers.l2(0.001)),
            layers.Dropout(0.2),
            layers.BatchNormalization(),
            layers.Dense(32, activation="swish", kernel_regularizer=keras.regularizers.l2(0.001)),
            layers.Dropout(0.1),
            layers.Dense(1, activation="sigmoid"),
        ],
        name="mlp_on_ae_features",
    )
    model_mlp_ae.summary()

    # --- モデルのコンパイル ---
    print("\n--- モデルコンパイル ---")
    model_mlp_ae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                         loss="binary_crossentropy",
                         metrics=[keras.metrics.AUC(name='auc'), 'accuracy'])

    # --- モデルの学習 ---
    print("\n--- モデル学習開始 (AE特徴量使用) ---")
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
    epochs = 100
    batch_size = 2048 # 前回と同じバッチサイズ

    start_train_time = time.time()
    history_mlp_ae = model_mlp_ae.fit(X_train_input, # AE特徴量を使用
                                      y_train_b,      # 元のターゲット変数
                                      batch_size=batch_size,
                                      epochs=epochs,
                                      validation_data=(X_test_input, y_test_b), # AE特徴量と元のターゲット
                                      callbacks=[early_stopping],
                                      verbose=1)
    end_train_time = time.time()
    print(f"学習完了。所要時間: {end_train_time - start_train_time:.2f} 秒")

    # --- テストデータでの最終評価 ---
    print("\n--- テストデータでの最終評価 (MLP on AE Features) ---")
    loss, auc_final_mlp_ae, accuracy_final_mlp_ae = model_mlp_ae.evaluate(X_test_input, y_test_b, verbose=0)

    print(f"テストデータ Loss: {loss:.4f}")
    print(f"テストデータ Accuracy: {accuracy_final_mlp_ae:.4f}")
    print(f"テストデータ AUC Score: {auc_final_mlp_ae:.4f}") # ★これが MLP on AE の性能

    # --- 予測確率の取得 (バックテスト用) ---
    print("\n--- 予測確率の取得 ---")
    y_pred_proba_mlp_ae = model_mlp_ae.predict(X_test_input).flatten()
    print("予測確率取得完了。")

    # (オプション) Classification Report と Confusion Matrix
    y_pred_mlp_ae = (y_pred_proba_mlp_ae > 0.5).astype(int)
    print("\nClassification Report (閾値0.5):")
    print(classification_report(y_test_b, y_pred_mlp_ae, target_names=['Low (0)', 'High (1)']))
    print("\nConfusion Matrix (閾値0.5):")
    conf_matrix_mlp_ae = confusion_matrix(y_test_b, y_pred_mlp_ae)
    conf_matrix_mlp_ae_df = pd.DataFrame(conf_matrix_mlp_ae, index=['Actual Low', 'Actual High'], columns=['Predicted Low', 'Predicted High'])
    display(conf_matrix_mlp_ae_df)

    # 結果を保存
    if 'model_results' not in locals(): model_results = {}
    model_results['MLP_on_AE_Features'] = {
        'model': model_mlp_ae,
        'auc': auc_final_mlp_ae,
        'accuracy': accuracy_final_mlp_ae,
        'y_pred_proba': y_pred_proba_mlp_ae,
        'features': 'Set B -> AE (dim=16)'
    }
    # モデルの保存 (任意)
    # model_mlp_ae.save('mlp_model_on_ae_features.keras')

else:
    print("AE特徴量 (`X_train_ae`, `X_test_ae`) またはターゲットデータ (`y_train_b`, `y_test_b`) が見つかりません。")


入力特徴量の数 (AE): 16

--- MLPモデル定義 (AE特徴量用) ---


Model: "mlp_on_ae_features"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization_4           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,377 (52.25 KB)

 Trainable params: 12,961 (50.63 KB)

 Non-trainable params: 416 (1.62 KB)


--- モデルコンパイル ---

--- モデル学習開始 (AE特徴量使用) ---
Epoch 1/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5144 - auc: 0.5182 - loss: 0.8611 - val_accuracy: 0.5104 - val_auc: 0.5162 - val_loss: 0.7709
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5280 - auc: 0.5376 - loss: 0.7561 - val_accuracy: 0.5119 - val_auc: 0.5189 - val_loss: 0.7253
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5313 - auc: 0.5435 - loss: 0.7168 - val_accuracy: 0.5154 - val_auc: 0.5238 - val_loss: 0.7056
Epoch 4/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5322 - auc: 0.5448 - loss: 0.7008 - val_accuracy: 0.5145 - val_auc: 0.5237 - val_loss: 0.6981
Epoch 5/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5321 - auc: 0.5457 - loss: 0.6946 - val_accuracy: 0.5160 - val_auc: 0.5250 - val_loss: 0.6949
Epoch 6/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5324 - auc: 0.5467 - loss: 0.6920 - val_accuracy: 0.5156 - val_auc: 0.5243 - val_lo

,Predicted Low,Predicted High
Actual Low,31122,22227
Actual High,29415,23937
